In [6]:
# import tensorflow as tf

def train():
    import tensorflow as tf
    
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    print("Training...")

    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 

    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [8]:
from kubeflow import fairing

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'

fairing.config.set_builder(
    'append',
    base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', 
    registry = PRIVATE_REGISTRY, 
    push=True
)

fairing.config.set_deployer('job')

remote_train = fairing.config.fn(train)

remote_train()

[I 201207 00:43:18 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7feec40a3d30>
[I 201207 00:43:18 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7feec40a3f28>
[I 201207 00:43:18 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7feec40a3e10>
[W 201207 00:43:18 append:50] Building image using Append builder...
[I 201207 00:43:18 base:107] Creating docker context: /tmp/fairing_context_3g9wdkt3
[W 201207 00:43:18 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201207 00:43:18 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/kf-base:latest'
[W 201207 00:43:18 append:54] Image successfully built in 0.04884943300021405s.
[W 201207 00:43:18 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fair

2020-12-07 00:43:21.637643: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-12-07 00:43:21.637759: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-12-07 00:43:21.637768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
11493376/11490434 [==============================] - 0s 0us/step
2020-12-07 00:43:23.317613: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or direct

[W 201207 00:43:43 job:173] Cleaning up job fairing-job-dwpv8...


### curl로 프라이빗 레지스트리 확인

In [3]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/_catalog

{"repositories":["fairing-job","kf-base","my-02-python-file-fairing","my-04-notebook-single-file-fairing-job"]}


In [4]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/fairing-job/tags/list

{"name":"fairing-job","tags":["F96CB04D","49BBF86C","3503F8D2","DFE72D4D","37E30F06","959B340A","391B4C7E","CE70263D","613C8372","660AE662","C5909B76","473CD9F8","68EA1BE0"]}


# Function Fairing 패키지 버전 이슈
Function Fairing 시 fairing SDK가 컨테이너 내부에서 아래와 같은 명령을 실행
```python
python /app/function_shim.py --serialized_fn_file /app/pickled_fn.p --python_version 3.6
```
결과는 에러.

다음과 같이 테스트 하여 실행 성공과 실패 케이스를 정리함

- 실행 실패
  - base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image  
  - base_image = 'tensorflow/tensorflow:latest-py3',   
  - base_image = 'tensorflow/tensorflow:2.1.2-gpu',    
  - Base Image에 pip install tensorflow-gpu   <- 최신 2.3.1 설치됨
- 실행 성공  
  - base_image = 'brightfly/kubeflow-jupyter-lab:tf2.0-gpu', <- 실마리
  - base_image = 'tensorflow/tensorflow:2.0.3-gpu-py3',
  - Base Image에 pip install tensorflow-gpu==2.0.0 